# 贝叶斯优化

本文主要记录[BayesianOptimization](https://github.com/fmfn/BayesianOptimization)库的使用。内容结构都与原文档一致。安装方法：

```Shell
conda install -c conda-forge bayesian-optimization
```

## 基本使用

贝叶斯优化是对计算耗费大的函数进行优化时一种常用的方法，这里先给出一个贝叶斯优化的示例程序：

In [2]:
from bayes_opt import BayesianOptimization
from bayes_opt.event import Events
from bayes_opt.util import load_logs


def black_box_function(x, y):
    """Function with unknown internals we wish to maximize.

    This is just serving as an example, for all intents and
    purposes think of the internals of this function, i.e.: the process
    which generates its output values, as unknown.
    """
    return -x ** 2 - (y - 1) ** 2 + 1


# Bounded region of parameter space
pbounds = {'x': (2, 4), 'y': (-3, 3)}

optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

optimizer.maximize(
    init_points=10,
    n_iter=3,
)

print(optimizer.max)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  1        | -7.135    |  2.834    |  1.322    |
|  2        | -7.78     |  2.0      | -1.186    |
|  3        | -16.13    |  2.294    | -2.446    |
|  4        | -8.341    |  2.373    | -0.9266   |
|  5        | -7.392    |  2.794    |  0.2329   |
|  6        | -7.069    |  2.838    |  1.111    |
|  7        | -6.412    |  2.409    |  2.269    |
|  8        | -3.223    |  2.055    |  1.023    |
|  9        | -7.455    |  2.835    |  0.3521   |
|  10       | -12.11    |  2.281    | -1.811    |
|  11       | -3.071    |  2.0      |  1.266    |
|  12       | -3.671    |  2.151    |  1.211    |
|  13       | -3.041    |  2.006    |  1.127    |
{'target': -3.041430390491576, 'params': {'x': 2.006340577135143, 'y': 1.1266012599960877}}


根据上面的代码可以看出直接在BayesianOptimization初始化时指定f为要优化的函数即可，生成 optimizer 即生成了贝叶斯优化类的对象，接下来调用它的maximize函数即可进行寻优。init_points从外围来看，是初始的迭代次数，后面n_iter是后来的迭代次数。

接着可以从 optimizer 对象输出中间的迭代结果。

In [2]:
print("-----------------------------各次迭代的结果------------------------------")
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

-----------------------------各次迭代的结果------------------------------
Iteration 0: 
	{'target': -7.135455292718879, 'params': {'x': 2.8340440094051482, 'y': 1.3219469606529488}}
Iteration 1: 
	{'target': -7.779531005607566, 'params': {'x': 2.0002287496346898, 'y': -1.1860045642089614}}
Iteration 2: 
	{'target': -16.134894722612252, 'params': {'x': 2.2935117816342263, 'y': -2.445968431387213}}
Iteration 3: 
	{'target': -8.340778037007604, 'params': {'x': 2.3725204227553416, 'y': -0.9266356377417138}}
Iteration 4: 
	{'target': -7.392279298427363, 'params': {'x': 2.79353494846134, 'y': 0.23290040402014167}}
Iteration 5: 
	{'target': -7.068843753868606, 'params': {'x': 2.8383890288065894, 'y': 1.1113170023805568}}
Iteration 6: 
	{'target': -6.412432296144895, 'params': {'x': 2.408904499463035, 'y': 2.268704618345673}}
Iteration 7: 
	{'target': -3.2226211374385345, 'params': {'x': 2.0547751863958523, 'y': 1.0228050610704136}}
Iteration 8: 
	{'target': -7.454735524570369, 'params': {'x': 2.8346

可以寻优过程中改变寻优边界，来重新继续寻优，还可以指定参数值

In [3]:
print("---------------------------------寻优过程中改变寻优边界-------------------------------------")
# 可以在寻优计算过程中改变寻优边界
optimizer.set_bounds(new_bounds={"x": (-2, 3)})

optimizer.maximize(
    init_points=0,
    n_iter=5,
)

---------------------------------寻优过程中改变寻优边界-------------------------------------
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  14       | -1.651    |  1.618    |  1.186    |
|  15       | -0.8391   |  1.356    |  1.023    |
|  16       | -0.2766   |  1.116    |  1.174    |
|  17       |  0.2046   |  0.8918   |  1.011    |
|  18       |  0.5723   |  0.6385   |  1.142    |


In [10]:
print("---------------------------------寻优过程中探索特定参数值-------------------------------------")
# 指定要探索的参数值，lazy表示下面maximize时执行
optimizer.probe(
    params={"x": 0.5, "y": 0.7},
    lazy=True,
)

print(optimizer.space.keys)
optimizer.probe(
    params=[-0.3, 0.1],
    lazy=True,
)
optimizer.maximize(init_points=0, n_iter=0)

---------------------------------寻优过程中探索特定参数值-------------------------------------
['x', 'y']
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  19       |  0.66     |  0.5      |  0.7      |
|  20       |  0.66     |  0.5      |  0.7      |
|  21       |  0.1      | -0.3      |  0.1      |


optimizer对象的寻优过程也可以记录下来，注意optimizer对象要先subscribe，然后再maximize才会记录下日志：

In [4]:
print("---------------------------------存储与加载寻优过程记录-------------------------------------")
# 用JSONLogger存储、加载计算过程
logger = JSONLogger(path="./logs.json")
optimizer.subscribe(Events.OPTMIZATION_STEP, logger)
# Results will be saved in ./logs.json
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

---------------------------------存储与加载寻优过程记录-------------------------------------
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  19       | -6.289    |  2.004    |  2.81     |
|  20       |  0.7889   | -0.4329   |  1.154    |
|  21       |  0.9917   |  0.007159 |  0.9089   |
|  22       |  0.9026   | -0.0179   |  1.312    |
|  23       |  0.9762   | -0.1296   |  1.084    |


可以看到，会生成一个logs.json文件。该文件可以在后续过程中加载：

In [6]:
print("--------------------加载之前存储在./logs.json中的计算过程------------------------")
new_optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds={"x": (-2, 2), "y": (-2, 2)},
    verbose=2,
    random_state=7,
)
print(len(new_optimizer.space))

logs=load_logs(new_optimizer, logs=["./logs.json"]);
print(logs)
print("New optimizer is now aware of {} points.".format(len(new_optimizer.space)))

new_optimizer.maximize(
    init_points=0,
    n_iter=10,
)

--------------------加载之前存储在./logs.json中的计算过程------------------------
0
New optimizer is now aware of 5 points.
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  1        | -1.334    | -0.1494   | -0.5203   |
|  2        | -2.454    | -1.567    |  2.0      |
|  3        |  0.3436   |  0.7821   |  0.7884   |
|  4        | -5.139    |  2.0      | -0.4626   |
|  5        |  0.674    |  0.508    |  1.261    |
|  6        |  0.1656   | -0.7715   |  0.5109   |
|  7        | -12.0     | -2.0      | -2.0      |
|  8        |  0.6889   | -0.08321  |  0.4485   |
|  9        |  0.8517   |  0.3511   |  0.8419   |
|  10       |  0.8457   | -0.3256   |  0.7802   |


## 高级用法

接下来是一些稍微高级一些的用法。

In [ ]:
"""advanced 操作"""

from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction


# Let's start by definying our function, bounds, and instanciating an optimization object.
def black_box_function(x, y):
    """定义目标函数"""
    return -x ** 2 - (y - 1) ** 2 + 1


# 实例化bo
optimizer = BayesianOptimization(
    f=None,
    pbounds={'x': (-2, 2), 'y': (-3, 3)},
    verbose=2,
    random_state=1,
)

# 使用效用函数
utility = UtilityFunction(kind="ucb", kappa=2.5, xi=0.0)

next_point_to_probe = optimizer.suggest(utility)
print("Next point to probe is:", next_point_to_probe)

target = black_box_function(**next_point_to_probe)
print("Found the target value to be:", target)

optimizer.register(
    params=next_point_to_probe,
    target=target,
)


# 下面是处理离散参数的手段。 “//”运算符是向下取整的操作
def func_with_discrete_params(x, y, d):
    # Simulate necessity of having d being discrete.
    assert type(d) == int

    return ((x + y + d) // (1 + d)) / (1 + (x + y) ** 2)


def function_to_be_optimized(x, y, w):
    """w是要离散化的，用int强行离散化了"""
    d = int(w)
    return func_with_discrete_params(x, y, d)


optimizer = BayesianOptimization(
    f=function_to_be_optimized,
    pbounds={'x': (-10, 10), 'y': (-10, 10), 'w': (0, 5)},
    verbose=2,
    random_state=1,
)

optimizer.maximize(alpha=1e-3)

# 调整高斯回归
optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds={'x': (-2, 2), 'y': (-3, 3)},
    verbose=2,
    random_state=1,
)
optimizer.maximize(
    init_points=1,
    n_iter=5,
    # What follows are GP regressor parameters
    alpha=1e-3,
    n_restarts_optimizer=5
)
optimizer.set_gp_params(normalize_y=True)

## 可视化过程

接下来可视化过程来看看如何进行优选的。

In [ ]:
"""bo优化过程的可视化"""
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import gridspec


def target(x):
    """在优化里，第一步都是优化目标的确定"""
    return np.exp(-(x - 2) ** 2) + np.exp(-(x - 6) ** 2 / 10) + 1 / (x ** 2 + 1)


x = np.linspace(-2, 10, 10000).reshape(-1, 1)
y = target(x)

plt.plot(x, y)
plt.show()

# 然后是构建BO对象，构建对象时需要目标函数，以及参数的取值范围，即定义要探索的参数空间.random_state是伪随机数生成器的种子
optimizer = BayesianOptimization(target, {'x': (-2, 10)}, random_state=27)

# maximize是执行寻优的函数，设置的参数包括随机探索的步数init_points,bo要执行的步数n_iter,以及提取函数权衡exploitation和exploration的参数kappa
optimizer.maximize(init_points=2, n_iter=0, kappa=5)


def posterior(optimizer, x_obs, y_obs, grid):
    """目标函数的函数（高斯过程）的后验分布"""
    optimizer._gp.fit(x_obs, y_obs)
    # 返回的mu和sigma应该是高斯分布的均值和方差平方根
    mu, sigma = optimizer._gp.predict(grid, return_std=True)
    return mu, sigma


def plot_gp(optimizer, x, y):
    """可视化高斯过程"""
    fig = plt.figure(figsize=(16, 10))
    steps = len(optimizer.space)
    fig.suptitle(
        'Gaussian Process and Utility Function After {} Steps'.format(steps),
        fontdict={'size': 30}
    )

    # 划分图形绘制的模块位置,2行1列
    gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1])
    # gs[0]函数寻优绘制到上面，另一个提取函数相关的绘制在下面。
    axis = plt.subplot(gs[0])
    acq = plt.subplot(gs[1])

    x_obs = np.array([[res["params"]["x"]] for res in optimizer.res])
    y_obs = np.array([res["target"] for res in optimizer.res])
    # 求出新的mu和sigma
    mu, sigma = posterior(optimizer, x_obs, y_obs, x)
    axis.plot(x, y, linewidth=3, label='Target')
    axis.plot(x_obs.flatten(), y_obs, 'D', markersize=8, label=u'Observations', color='r')
    axis.plot(x, mu, '--', color='k', label='Prediction')
    # 绘制%95置信度的空间，就是均值附近1.96倍sigma
    axis.fill(np.concatenate([x, x[::-1]]),
              np.concatenate([mu - 1.9600 * sigma, (mu + 1.9600 * sigma)[::-1]]),
              alpha=.6, fc='c', ec='None', label='95% confidence interval')
    # 坐标方面相关的设置
    axis.set_xlim((-2, 10))
    axis.set_ylim((None, None))
    axis.set_ylabel('f(x)', fontdict={'size': 20})
    axis.set_xlabel('x', fontdict={'size': 20})
    # 效用函数（也就是acquisition function），用的是ucb，这个在几个常用的提取函数里简单，但是却比较好用.ucb用搞不到xi
    utility_function = UtilityFunction(kind="ucb", kappa=5, xi=0)
    # 调用效益函数进行计算
    utility = utility_function.utility(x, optimizer._gp, 0)
    acq.plot(x, utility, label='Utility Function', color='purple')
    acq.plot(x[np.argmax(utility)], np.max(utility), '*', markersize=15,
             label=u'Next Best Guess', markerfacecolor='gold', markeredgecolor='k', markeredgewidth=1)
    acq.set_xlim((-2, 10))
    acq.set_ylim((0, np.max(utility) + 0.5))
    acq.set_ylabel('Utility', fontdict={'size': 20})
    acq.set_xlabel('x', fontdict={'size': 20})

    axis.legend(loc=2, bbox_to_anchor=(1.01, 1), borderaxespad=0.)
    acq.legend(loc=2, bbox_to_anchor=(1.01, 1), borderaxespad=0.)


# 每一次优化都可视化其优化过程
plot_gp(optimizer, x, y)
plt.show()

optimizer.maximize(init_points=0, n_iter=1, kappa=5)
plot_gp(optimizer, x, y)
plt.show()

optimizer.maximize(init_points=0, n_iter=1, kappa=5)
plot_gp(optimizer, x, y)
plt.show()

optimizer.maximize(init_points=0, n_iter=1, kappa=5)
plot_gp(optimizer, x, y)
plt.show()

optimizer.maximize(init_points=0, n_iter=1, kappa=5)
plot_gp(optimizer, x, y)
plt.show()

## exploration 与  exploitation 

这是贝叶斯超参数优化中的两个重要术语。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from bayes_opt import BayesianOptimization

np.random.seed(42)
xs = np.linspace(-2, 10, 10000)


def f(x):
    """定义目标函数"""
    return np.exp(-(x - 2) ** 2) + np.exp(-(x - 6) ** 2 / 10) + 1 / (x ** 2 + 1)


plt.plot(xs, f(xs))
plt.show()


def plot_bo(f, bo):
    """绘制贝叶斯优化过程"""
    x = np.linspace(-2, 10, 10000)
    mean, sigma = bo._gp.predict(x.reshape(-1, 1), return_std=True)

    plt.figure(figsize=(16, 9))
    plt.plot(x, f(x))
    plt.plot(x, mean)
    plt.fill_between(x, mean + sigma, mean - sigma, alpha=0.1)
    plt.scatter(bo.space.params.flatten(), bo.space.target, c="red", s=50, zorder=10)
    plt.show()


# 构造BO对象
bo = BayesianOptimization(
    f=f,
    pbounds={"x": (-2, 10)},
    verbose=0,
    random_state=987234,
)
# 设置一个kappa值，小的值是偏向于exploitation，即在现有值附近探索，保证更大
bo.maximize(n_iter=10, acq="ucb", kappa=0.1)

plot_bo(f, bo)

bo = BayesianOptimization(
    f=f,
    pbounds={"x": (-2, 10)},
    verbose=0,
    random_state=987234,
)
# 设置新的较大kappa值，尽可能地探索未知的空间
bo.maximize(n_iter=10, acq="ucb", kappa=10)

plot_bo(f, bo)

bo = BayesianOptimization(
    f=f,
    pbounds={"x": (-2, 10)},
    verbose=0,
    random_state=987234,
)

# 换一个提取函数，换为EI方法。xi小的时候，偏向于exploitation
bo.maximize(n_iter=10, acq="ei", xi=1e-4)

plot_bo(f, bo)

bo = BayesianOptimization(
    f=f,
    pbounds={"x": (-2, 10)},
    verbose=0,
    random_state=987234,
)
# 偏大的时候，偏向于探索exploration
bo.maximize(n_iter=10, acq="ei", xi=1e-1)

plot_bo(f, bo)

bo = BayesianOptimization(
    f=f,
    pbounds={"x": (-2, 10)},
    verbose=0,
    random_state=987234,
)
# 常用的另一种方法，poi法。同样，小的xi篇exploition，大的偏exploration
bo.maximize(n_iter=10, acq="poi", xi=1e-4)

plot_bo(f, bo)

bo = BayesianOptimization(
    f=f,
    pbounds={"x": (-2, 10)},
    verbose=0,
    random_state=987234,
)

bo.maximize(n_iter=10, acq="poi", xi=1e-1)

plot_bo(f, bo)

## 贝叶斯优化实例

最后是一个用交叉验证和贝叶斯优化对机器学习模型SVR和RF调参的例子。

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import SVC

from bayes_opt import BayesianOptimization
from bayes_opt.util import Colours


def get_data():
    """Synthetic binary classification dataset."""
    data, targets = make_classification(
        n_samples=1000,
        n_features=45,
        n_informative=12,
        n_redundant=7,
        random_state=134985745,
    )
    return data, targets


def svc_cv(C, gamma, data, targets):
    """SVC cross validation.
    This function will instantiate a SVC classifier with parameters C and
    gamma. Combined with data and targets this will in turn be used to perform
    cross validation. The result of cross validation is returned.
    Our goal is to find combinations of C and gamma that maximizes the roc_auc
    metric.
    """
    estimator = SVC(C=C, gamma=gamma, random_state=2)
    cval = cross_val_score(estimator, data, targets, scoring='roc_auc', cv=4)
    return cval.mean()


def rfc_cv(n_estimators, min_samples_split, max_features, data, targets):
    """Random Forest cross validation.
    This function will instantiate a random forest classifier with parameters
    n_estimators, min_samples_split, and max_features. Combined with data and
    targets this will in turn be used to perform cross validation. The result
    of cross validation is returned.
    Our goal is to find combinations of n_estimators, min_samples_split, and
    max_features that minimzes the log loss.
    """
    estimator = RFC(
        n_estimators=n_estimators,
        min_samples_split=min_samples_split,
        max_features=max_features,
        random_state=2
    )
    cval = cross_val_score(estimator, data, targets,
                           scoring='neg_log_loss', cv=4)
    return cval.mean()


def optimize_svc(data, targets):
    """Apply Bayesian Optimization to SVC parameters."""

    def svc_crossval(expC, expGamma):
        """Wrapper of SVC cross validation.
        Notice how we transform between regular and log scale. While this
        is not technically necessary, it greatly improves the performance
        of the optimizer.
        """
        C = 10 ** expC
        gamma = 10 ** expGamma
        return svc_cv(C=C, gamma=gamma, data=data, targets=targets)

    optimizer = BayesianOptimization(
        f=svc_crossval,
        pbounds={"expC": (-3, 2), "expGamma": (-4, -1)},
        random_state=1234,
        verbose=2
    )
    optimizer.maximize(n_iter=10)

    print("Final result:", optimizer.max)


def optimize_rfc(data, targets):
    """Apply Bayesian Optimization to Random Forest parameters."""

    def rfc_crossval(n_estimators, min_samples_split, max_features):
        """Wrapper of RandomForest cross validation.
        Notice how we ensure n_estimators and min_samples_split are casted
        to integer before we pass them along. Moreover, to avoid max_features
        taking values outside the (0, 1) range, we also ensure it is capped
        accordingly.
        """
        return rfc_cv(
            n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=max(min(max_features, 0.999), 1e-3),
            data=data,
            targets=targets,
        )

    optimizer = BayesianOptimization(
        f=rfc_crossval,
        pbounds={
            "n_estimators": (10, 250),
            "min_samples_split": (2, 25),
            "max_features": (0.1, 0.999),
        },
        random_state=1234,
        verbose=2
    )
    optimizer.maximize(n_iter=10)

    print("Final result:", optimizer.max)


if __name__ == "__main__":
    data, targets = get_data()

    print(Colours.yellow("--- Optimizing SVM ---"))
    optimize_svc(data, targets)

    print(Colours.green("--- Optimizing Random Forest ---"))
    optimize_rfc(data, targets)